In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '2,3,4,7'

In [2]:
from src.ModelTransfuser import ModelTransfuser as MTf

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import torch

from scipy.stats import norm, gaussian_kde
from scipy import optimize

import os

%matplotlib inline

# Load data

In [3]:
data_model = "842"

In [4]:
# --- Clean the data ---
# Chempy sometimes returns zeros or infinite values, which need to removed
def clean_data(x, y):
    # Remove all zeros from the training data
    index = np.where((y == 0).all(axis=1))[0]
    x = np.delete(x, index, axis=0)
    y = np.delete(y, index, axis=0)

    # Remove all infinite values from the training data
    index = np.where(np.isfinite(y).all(axis=1))[0]
    x = x[index]
    y = y[index]

    # Remove H from Elements
    y = np.delete(y, 2, 1)

    return x, y

In [5]:
# ---  Load in the validation data ---
path_test = os.getcwd() + f'/data/Chempy_model_comp_data/chempy_{data_model}_val.npz'
val_data = np.load(path_test, mmap_mode='r')

val_x = val_data['params']
val_y = val_data['abundances']

val_x, val_y = clean_data(val_x, val_y)

# convert to torch tensors
val_x = torch.tensor(val_x, dtype=torch.float32)
val_y = torch.tensor(val_y, dtype=torch.float32)

# --- add noise ---
pc_ab = 5 # percentage error in abundance

val_y_err = torch.ones_like(val_y)*float(pc_ab)/100.
val_y = norm.rvs(loc=val_y,scale=val_y_err)
val_y = torch.tensor(val_y).float()

val_data = torch.cat((val_x, val_y), 1)

## Define ModelTransfuser

In [6]:
# Define the ModelTransfuser
model_442 = MTf.load('data/models/chempy_442/Model_checkpoint.pt')
model_742 = MTf.load('data/models/chempy_742/Model_checkpoint.pt')
model_842 = MTf.load('data/models/chempy_842/Model_checkpoint.pt')

## Sampling

In [7]:
posterior_mask = torch.zeros_like(val_data[0])
posterior_mask[6:] = 1

data = val_data[:1000, posterior_mask.type(torch.bool)]

In [8]:
test_index=1

In [9]:
posterior_442 = model_442.sample(data[test_index], condition_mask=posterior_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
posterior_742 = model_742.sample(data[test_index], condition_mask=posterior_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
posterior_842 = model_842.sample(data[test_index], condition_mask=posterior_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")

100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


In [10]:
theta_hat_442 = posterior_442.mean(dim=1)[:,:6]
theta_hat_742 = posterior_742.mean(dim=1)[:,:6]
theta_hat_842 = posterior_842.mean(dim=1)[:,:6]

In [11]:
print(f"Ground Truth: {val_data[test_index,:6]}")
print(f"Model 442: {theta_hat_442}")
print(f"Model 742: {theta_hat_742}")
print(f"Model 842: {theta_hat_842}")

Ground Truth: tensor([-2.6000, -3.0300, -0.2485,  0.5487,  0.6439,  4.7983])
Model 442: tensor([[-2.4927, -3.2242, -0.1933,  0.5822,  0.5195,  3.3998]])
Model 742: tensor([[-1.6766, -3.3914, -0.1068,  0.6421,  0.5726,  3.3389]])
Model 842: tensor([[-2.5692, -2.9983, -0.3979,  0.5642,  0.5260,  4.3287]])


In [12]:
likelihood_mask = torch.zeros_like(val_data[0])
likelihood_mask[:6] = 1

In [13]:
likelihood_442 = model_442.sample(theta_hat_442, condition_mask=likelihood_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
likelihood_742 = model_742.sample(theta_hat_742, condition_mask=likelihood_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")
likelihood_842 = model_842.sample(theta_hat_842, condition_mask=likelihood_mask, device="cuda:0", timesteps=100, cfg_alpha=None, save_trajectory=True, corrector_steps=5, corrector_steps_interval=5, final_corrector_steps=10).to("cpu")

100%|██████████| 1/1 [00:01<00:00,  1.71s/it]


In [14]:
x_hat_442 = likelihood_442.mean(dim=1)[:,6:]
x_hat_742 = likelihood_742.mean(dim=1)[:,6:]
x_hat_842 = likelihood_842.mean(dim=1)[:,6:]

In [15]:
print(f"Ground Truth: {val_data[test_index,6:]}")
print(f"Model 442: {x_hat_442}")
print(f"Model 742: {x_hat_742}")
print(f"Model 842: {x_hat_842}")

Ground Truth: tensor([ 0.0819, -0.5307,  0.4654,  0.3000,  0.3074,  0.3703,  0.4098,  0.4402])
Model 442: tensor([[ 0.2086, -0.3852,  0.3975,  0.1628,  0.2034,  0.5245,  0.3849,  0.2844]])
Model 742: tensor([[-0.0600, -0.2195,  0.3048, -0.0178,  0.2267,  0.1952,  0.3454,  0.2704]])
Model 842: tensor([[-0.0087, -0.5164,  0.5212,  0.2097,  0.2341,  0.4781,  0.3287,  0.2987]])


In [16]:
likelihood_fn_442 = gaussian_kde(likelihood_442[0,:,6:].T.cpu().numpy())
likelihood_fn_742 = gaussian_kde(likelihood_742[0,:,6:].T.cpu().numpy())
likelihood_fn_842 = gaussian_kde(likelihood_842[0,:,6:].T.cpu().numpy())

In [17]:
likelihood_fn_442.pdf(data[test_index])

array([157.77525158])

In [18]:
likelihood_fn_742.pdf(data[test_index])

array([15.39868265])

In [19]:
likelihood_fn_842.pdf(data[test_index])

array([586.79256383])